# Asynchronous Programming in Python

Python, by default, operates in a single thread due to the Global Interpreter Lock (GIL). This design means that asynchronous programming in Python is most effective for I/O-bound tasks. For example, if your program needs to download a large volume of data over the internet, asynchronous programming allows it to perform other operations (such as updating the UI or processing data) while waiting for the network response.

However, for CPU-bound tasks—where the primary workload is intensive computations—multi-threading (or even multiprocessing) might be more appropriate. This is because asynchronous programming doesn't provide significant benefits when the bottleneck is the CPU rather than waiting on I/O operations.

In [31]:
import aiohttp
import asyncio

async def fetch_large_file(url):
    """Fetch a large file from the given URL and return its content as text."""
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            # We can stream the response in chunks to avoid loading everything into memory at once.
            data = bytearray()
            async for chunk in response.content.iter_chunked(1):  # 1024 bytes per chunk
                data.extend(chunk)
            return data.decode('utf-8')

async def read_file():
    url = "https://norvig.com/big.txt"
    print("Downloading large data file...")
    content = await fetch_large_file(url)
    print("Download complete!")
    print("Total characters downloaded:", len(content))
    return content[0:100]
    # Further processing can be done here

async def say_after(delay, message):
    # Simulate an I/O-bound operation with asyncio.sleep
    await asyncio.sleep(delay)
    print(message)
    return message

async def main():
    task1 = asyncio.create_task(read_file())
    task2 = asyncio.create_task(say_after(0,"I'm waiting"))
    content, msg = await asyncio.gather(task1, task2)
    print(msg)
    
await main()


I'm waiting
Download complete!
Total characters downloaded: 6488666
I'm waiting


One thing to note is how the main function handles the tasks. When you use `content, msg = await asyncio.gather(task1, task2)`, the call to `asyncio.gather()` waits for both tasks to finish before returning their results. This means that even if the say_after function completes (and its result is ready) before read_file finishes downloading, the result of say_after won’t be printed until after both tasks have completed. In other words, while say_after may finish early, the `await asyncio.gather(...)` call delays processing until all tasks are done. If you want to see the "I'm waiting" message immediately when say_after finishes, you could either print it directly inside say_after or use `asyncio.as_completed()` to handle tasks as they complete.

Lets look at using `asyncio.as_completed()`

In [32]:
async def say_after(delay, message):
    # Simulate an I/O-bound operation with asyncio.sleep
    await asyncio.sleep(delay)
    #print(message) - removing print so we can see the change
    return message

async def main():
    tasks = [asyncio.create_task(read_file()), 
             asyncio.create_task(say_after(0, "I'm waiting"))]
    
    for completed in asyncio.as_completed(tasks):
        result = await completed
        print(result)

await main()

I'm waiting
Download complete!
Total characters downloaded: 6488666
The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in o
